# Lab | Summarization evaluation using LangSmith
Let's revisit your capstone project 2? Well, sort of. Pick diffierent sets of data and re-run this notebook. Maybe parts of the dataset you used in your last project week. The point is for you to understand all steps involve and the many different ways one can and should evaluate LLM applications using LangSmith.

What did you learn? - Let's discuss that in class

## LangSmith - LangChain evaluation

In [9]:
!pip install python-dotenv


In [57]:
# from dotenv import load_dotenv, find_dotenv
# import os

# _ = load_dotenv(find_dotenv())

# # ⌨️ Input boxes to enter the keys
# OPENAI_API_KEY = input("Enter your OpenAI API key: ")
# LANGCHAIN_API_KEY = input("Enter your LangChain API key: ")
# HUGGINGFACEHUB_API_TOKEN = input("Enter your HuggingFace API token: ")

# # Store them in environment variables
# os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
# os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY
# os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN


In [11]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"]="https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"]="langsmith_max-test"

In [12]:
#Importing Client from Langsmith
from langsmith import Client
client = Client(api_key=LANGCHAIN_API_KEY)

### Create Dataset


In [14]:
!pip install datasets  # ✅ install required library

from datasets import load_dataset
cnn_dataset = load_dataset(
    "ccdv/cnn_dailymail", version="3.0.0", trust_remote_code=True
)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == 

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/13.9k [00:00<?, ?B/s]

cnn_dailymail.py:   0%|          | 0.00/9.27k [00:00<?, ?B/s]

cnn_stories.tgz:   0%|          | 0.00/159M [00:00<?, ?B/s]

dailymail_stories.tgz:   0%|          | 0.00/376M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
def add_prefix(example):
    return {
        **example,
        "article": f"Summarize this news:\n{example['article']}"
    }

#cnn_dataset = cnn_dataset.map(add_prefix)

In [ ]:
cnn_dataset

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})

In [15]:
cnn_dataset['train'][0]

{'article': 'It\'s official: U.S. President Barack Obama wants lawmakers to weigh in on whether to use military force in Syria. Obama sent a letter to the heads of the House and Senate on Saturday night, hours after announcing that he believes military action against Syrian targets is the right step to take over the alleged use of chemical weapons. The proposed legislation from Obama asks Congress to approve the use of military force "to deter, disrupt, prevent and degrade the potential for future uses of chemical weapons or other weapons of mass destruction." It\'s a step that is set to turn an international crisis into a fierce domestic political battle. There are key questions looming over the debate: What did U.N. weapons inspectors find in Syria? What happens if Congress votes no? And how will the Syrian government react? In a televised address from the White House Rose Garden earlier Saturday, the president said he would take his case to Congress, not because he has to -- but bec

In [17]:
#Get just a few news to test
def add_prefix(example):
    example["article"] = "News: " + example["article"]
    example["highlights"] = "Summary: " + example["highlights"]
    return example


The dataset contains three columns: article, highlights, and id. To use LangSmith, we need to create a dataset in LangSmith format.

LangSmith expects a prompt and a result. To achieve this, we will transform the article into a prompt by adding the prefix: "Summarize this news." As a result, we will use the content of highlights, which represents the summaries created by humans.

In [19]:

MAX_NEWS = 10
sample_cnn = cnn_dataset["test"].select(range(MAX_NEWS)).map(add_prefix)


Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [20]:
print(sample_cnn[0])


{'article': 'News: (CNN)James Best, best known for his portrayal of bumbling sheriff Rosco P. Coltrane on TV\'s "The Dukes of Hazzard," died Monday after a brief illness. He was 88. Best died in hospice in Hickory, North Carolina, of complications from pneumonia, said Steve Latshaw, a longtime friend and Hollywood colleague. Although he\'d been a busy actor for decades in theater and in Hollywood, Best didn\'t become famous until 1979, when "The Dukes of Hazzard\'s" cornpone charms began beaming into millions of American homes almost every Friday night. For seven seasons, Best\'s Rosco P. Coltrane chased the moonshine-running Duke boys back and forth across the back roads of fictitious Hazzard County, Georgia, although his "hot pursuit" usually ended with him crashing his patrol car. Although Rosco was slow-witted and corrupt, Best gave him a childlike enthusiasm that got laughs and made him endearing. His character became known for his distinctive "kew-kew-kew" chuckle and for goofy c

Now We have the Dataset with the prompt and the Reference Summary, it is time to create a Dataset in LangSmith with this information.
### Create the Dataset in Langsmith

The dataset in LangSmith is composed of an input, which is the prompt passed to the model for evaluation, and an output, which should contain what we expect the model to return.

In [21]:
import datetime

In [22]:
import uuid
input_key=['article']
output_key=['highlights']

NAME_DATASET=f"Summarize_dataset_{datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"

In [23]:
#This creates the dataset in LangSmith with the content in sample_cnn - If you run this more than once you will get POST errors
dataset = client.upload_dataframe(
    df=sample_cnn,
    input_keys=input_key,
    output_keys=output_key,
    name=NAME_DATASET,
    description="Test Embedding distance between model summarizations",
    data_type="kv"
)

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

In this image, we can see an example from the dataset once it's been registered in LangSmith.

In the Input column, there is the prompt to be sent, while in the Output column, the expected output is stored.

When performing the comparison, the model will be given the prompt, and the Cosine distance between its response and the one stored in the sample dataset will be calculated.
<img src="https://github.com/peremartra/Large-Language-Model-Notebooks-Course/blob/main/img/Martra_Figure_4_2SDL_Dataset.jpg?raw=true">

### Recovering Models From Hugging Face
Let's retrieve both models from HuggingFace. A base T5 model and a model that has been fine-tuned using the training portion of this same dataset to generate summaries.

In [25]:
# ✅ Install required dependencies
!pip install langchain langchain-community

# ✅ Your original import (unchanged)
from langchain import HuggingFaceHub


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 79.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.0 MB/s eta 0:00:00


In [38]:
!pip install transformers

from transformers import pipeline

# Load summarization pipeline
summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6")

# Your input text
input_text = "The Titanic was a British passenger liner that sank in 1912 after hitting an iceberg. It was one of the deadliest disasters in maritime history."

# Generate summary
summary = summarizer(input_text, max_length=180, min_length=30, do_sample=False)

# Output result
print("📄 Original Text:\n", input_text)
print("\n📝 Summary:\n", summary[0]['summary_text'])


config.json:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

Device set to use cpu
Your max_length is set to 180, but your input_length is only 29. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=14)


📄 Original Text:
 The Titanic was a British passenger liner that sank in 1912 after hitting an iceberg. It was one of the deadliest disasters in maritime history.

📝 Summary:
  Titanic was a British passenger liner that sank in 1912 after hitting an iceberg . It was one of the deadliest disasters in maritime history . The Titanic was the largest passenger liner to sink in history .


In [43]:
from getpass import getpass
import os

# ✅ Step 1: Enter the API token in the output input box
HUGGINGFACEHUB_API_TOKEN = getpass("🔐 Enter your Hugging Face API token:")
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN

# ✅ Step 2: Your original code unchanged
from langchain import HuggingFaceHub  # or from langchain_community if needed

summarizer_finetuned = HuggingFaceHub(
    repo_id="flax-community/t5-base-cnn-dm",
    model_kwargs={"temperature": 0, "max_length": 180},
    huggingfacehub_api_token=HUGGINGFACEHUB_API_TOKEN
)


🔐 Enter your Hugging Face API token:··········


## Defining Evaluator
The first step is to define an evaluator, where we specify the variables we want to evaluate. In our case, I have chosen to measure only the "embedding_distance."

I've left the "string_distance" as a comment in case you want to conduct a test with two evaluations instead of one.


In [47]:
from langchain.smith import run_on_dataset, RunEvalConfig
!pip install -q rapidfuzz==3.6.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 43.7 MB/s eta 0:00:00


In [48]:
#We are using just one of the multiple evaluator avaiable on LangSmith.

evaluation_config = RunEvalConfig(
    evaluators=[
        "embedding_distance",
        #"string_distance"
    ],
)



### Running Evaluator
With the same configuration, we can launch two evaluations on the same dataset. One for each of the chosen models.

In [50]:
!pip install tiktoken


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 48.5 MB/s eta 0:00:00


In [51]:
project_name = f"T5-BASE {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"

base_t5_results = run_on_dataset(
    client=client,
    project_name=project_name,
    dataset_name=NAME_DATASET,
    llm_or_chain_factory=summarizer_base,
    evaluation=evaluation_config,
)

View the evaluation results for project 'T5-BASE 2025-04-22 12:00:56' at:
https://smith.langchain.com/o/2e586224-280b-450c-92e5-e2659fd69f73/datasets/0a3b1df7-9e32-4df1-b1cf-5ee07cb601dc/compare?selectedSessions=7f160dbd-b553-4281-8e82-a1c694b0b5db

View all tests for Dataset Summarize_dataset_2025-04-22 11:08:27 at:
https://smith.langchain.com/o/2e586224-280b-450c-92e5-e2659fd69f73/datasets/0a3b1df7-9e32-4df1-b1cf-5ee07cb601dc
[>                                                 ] 0/10

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_

[--------->                                        ] 2/10

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)
Error Type: BadRequestError, Message: (Request ID: Root=1-680784f9-7440b48b694c3b0e3b0dc0cf;0f9b478a-4fbe-479d-a2a4-8930147fdbb0)

Bad request:
The following `model_kwargs` are not used by the model: ['stop_sequences', 'watermark', 'return_full_text', 'stop'] (note: typos in the generate arguments will also show up in this list)
Error Type: BadRequestError, Message: (Request ID: Root=1-680784f9-381e569915de08f07b407319;aeaf9a5c-fdc2-480d-aa73-033652743081)

Bad request:
The following

[------------------->                              ] 4/10

...

[------------------------>                         ] 5/10

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)
Error Type: BadRequestError, Message: (Request ID: Root=1-680784fa-42ac6ef21247bbf12b3b0aea;1da48e6e-1514-47b9-b425-ef0a77a37f36)

Bad request:
The following `model_kwargs` are not used by the model: ['stop_sequences', 'watermark', 'return_full_text', 'stop'] (note: typos in the generate arguments will also show up in this list)
Error Type: BadRequestError, Message: (Request ID: Root=1-680784fa-4219e8493abf737f012cdc8f;d4b67b66-f911-457f-9781-fd916b3ddee8)

Bad request:
The following

[-------------------------------------------->     ] 9/10

Error Type: BadRequestError, Message: (Request ID: Root=1-680784fa-5ce0b739524263b042c99e4a;d3c93524-a937-47ce-b3a2-4abe3b8bd005)

Bad request:
The following `model_kwargs` are not used by the model: ['watermark', 'stop_sequences', 'stop', 'return_full_text'] (note: typos in the generate arguments will also show up in this list)


[------------------------------------------------->] 10/10

In [52]:
#Ignore the error shown below
project_name = f"T5-FineTuned {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"

finetuned_t5_results = run_on_dataset(
    client=client,
    project_name=project_name,
    dataset_name=NAME_DATASET,
    llm_or_chain_factory=summarizer_finetuned,
    evaluation=evaluation_config,
)

View the evaluation results for project 'T5-FineTuned 2025-04-22 12:01:05' at:
https://smith.langchain.com/o/2e586224-280b-450c-92e5-e2659fd69f73/datasets/0a3b1df7-9e32-4df1-b1cf-5ee07cb601dc/compare?selectedSessions=2d337b36-1789-42ed-b1e2-ed57fe667a0e

View all tests for Dataset Summarize_dataset_2025-04-22 11:08:27 at:
https://smith.langchain.com/o/2e586224-280b-450c-92e5-e2659fd69f73/datasets/0a3b1df7-9e32-4df1-b1cf-5ee07cb601dc
[>                                                 ] 0/10

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_

[-------------->                                   ] 3/10

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)
Error Type: HfHubHTTPError, Message: 503 Server Error: Service Temporarily Unavailable for url: https://router.huggingface.co/hf-inference/models/flax-community/t5-base-cnn-dm


[------------------->                              ] 4/10

...

[--------------------------------------->          ] 8/10

Error Type: HfHubHTTPError, Message: 503 Server Error: Service Temporarily Unavailable for url: https://router.huggingface.co/hf-inference/models/flax-community/t5-base-cnn-dm


[-------------------------------------------->     ] 9/10

Error Type: HfHubHTTPError, Message: 503 Server Error: Service Temporarily Unavailable for url: https://router.huggingface.co/hf-inference/models/flax-community/t5-base-cnn-dm


[------------------------------------------------->] 10/10

<img src="https://github.com/peremartra/Large-Language-Model-Notebooks-Course/blob/main/img/Martra_Figure_4_2SDL_Tests.jpg?raw=true">

In the image below you can see the comparision between two tests.
<img src="https://github.com/peremartra/Large-Language-Model-Notebooks-Course/blob/main/img/Martra_Figure_4_2SDL_CompareTestst.jpg?raw=true">

Well, since it has been so straightforward, why don't we try to make the comparison with an OpenAI model?

In [54]:
!pip install langchain-openai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.1/434.1 kB 21.4 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.52
    Uninstalling langchain-core-0.3.52:
      Successfully uninstalled langchain-core-0.3.52


In [55]:
from langchain_openai import OpenAI
open_aillm=OpenAI(temperature=0.0)

In [56]:
project_name = f"OpenAI {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"

finetuned_t5_results = run_on_dataset(
    client=client,
    project_name=project_name,
    dataset_name=NAME_DATASET,
    llm_or_chain_factory=open_aillm,
    evaluation=evaluation_config,
)

View the evaluation results for project 'OpenAI 2025-04-22 12:02:33' at:
https://smith.langchain.com/o/2e586224-280b-450c-92e5-e2659fd69f73/datasets/0a3b1df7-9e32-4df1-b1cf-5ee07cb601dc/compare?selectedSessions=f9c2df38-b811-4476-8b51-411665838104

View all tests for Dataset Summarize_dataset_2025-04-22 11:08:27 at:
https://smith.langchain.com/o/2e586224-280b-450c-92e5-e2659fd69f73/datasets/0a3b1df7-9e32-4df1-b1cf-5ee07cb601dc
[------------------------------------------------->] 10/10

<img src="https://github.com/peremartra/Large-Language-Model-Notebooks-Course/blob/main/img/Martra_Figure_4_2SDL_CompareOpenAI_HF.jpg?raw=true">

The experiment with the OpenAI model has yielded the best results. But, be aware! As we can see, there is a cost involved since we are using an API, and it needs to be paid for.

Another crucial piece of information is that we can view performance data for the models. This data could also be useful for minimally evaluating our inference server.

Conclusion:

Based on the evaluation results, the OpenAI model demonstrated the best performance among all tested models. Its ability to generate high-quality summaries positions it as a strong candidate for production use.

Recommendation:

While the OpenAI API provides superior results, it is important to consider the associated costs. If budget constraints are a concern, alternative open-source models may be explored for a cost-efficient solution.
Additionally, leveraging the performance metrics provided by LangChain can help monitor and evaluate the efficiency of any deployed inference server over time.

